In [1]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns 
import matplotlib.pyplot as plt

%matplotlib notebook

In [34]:
data = pd.read_excel("临床问卷数据1.xlsx").iloc[:, :15]
data.loc[data['椎间盘退变分数']<=10, ['椎间盘退变分数']] = 0
data.loc[data['椎间盘退变分数']>10, ['椎间盘退变分数']] = 1
data["intercept"] = 1.0

In [35]:
data.head()

,年龄,身高,体重,BMI,吸烟史,饮酒史,体育锻炼,腰背痛程度评分,VAS评分,椎间盘退变分数,Modic改变阴性椎体个数,Modic改变阳性椎体个数,许莫氏结节阴性椎体个数,许莫氏结节阳性椎体个数,睡眠量表得分,intercept
0,35,160,53,20.703125,无,无,≥1次/周,14,5,1,4,1,5,0,70,1.0
1,32,175,80,26.122449,有,有,≥1次/周,26,6,1,1,4,4,1,59,1.0
2,33,170,60,20.761246,无,无,无,7,3,1,2,3,4,1,57,1.0
3,28,175,83,27.102041,戒,有,＜1次/周,11,2,1,0,5,2,3,40,1.0
4,32,183,80,23.888441,戒,有,＜1次/周,10,2,0,0,5,3,2,51,1.0


In [36]:
data['吸烟史'].unique(), data['饮酒史'].unique(), data['体育锻炼'].unique()

(array(['无', '有', '戒'], dtype=object),
 array(['无', '有'], dtype=object),
 array(['≥1次/周', '无', '＜1次/周'], dtype=object))

In [37]:
dummy_smoke = pd.get_dummies(data['吸烟史'], prefix='smoke', drop_first=True) # 无，有，戒 :0，1，2
dummy_alcohol = pd.get_dummies(data['饮酒史'], prefix='alcohol', drop_first=True)  # 无，有
dummy_exercise = pd.get_dummies(data['体育锻炼'], prefix='exercise', drop_first=True) # 无，》=， 《=

In [38]:
cols = ['年龄', '身高', '体重', 'BMI', '腰背痛程度评分', 'VAS评分','椎间盘退变分数', 
        'Modic改变阴性椎体个数', 'Modic改变阳性椎体个数', '许莫氏结节阴性椎体个数',
        '许莫氏结节阳性椎体个数', '睡眠量表得分', 'intercept']
df = data[cols].join(dummy_smoke).join(dummy_alcohol).join(dummy_exercise)

In [41]:
x_cols = ['intercept','年龄', '身高', '体重', 'BMI', '腰背痛程度评分', 'VAS评分',  '睡眠量表得分', 
       'smoke_无', 'smoke_有', 'alcohol_有', 'exercise_无', 'exercise_＜1次/周']
y_cols = ['椎间盘退变分数']

In [53]:
data['吸烟史'] = data['吸烟史'].map({'无':0, '有':1, '戒':2})
data['饮酒史'] = data['饮酒史'].map({'无':0, '有':1})
data['体育锻炼'] = data['体育锻炼'].map({'≥1次/周':1, '无':0, '＜1次/周':2})


In [7]:
data.head()

,年龄,身高,体重,BMI,吸烟史,饮酒史,体育锻炼,腰背痛程度评分,VAS评分,椎间盘退变分数,Modic改变阴性椎体个数,Modic改变阳性椎体个数,许莫氏结节阴性椎体个数,许莫氏结节阳性椎体个数,睡眠量表得分,intercept
0,35,160,53,20.703125,0,0,1,14,5,1,4,1,5,0,70,1.0
1,32,175,80,26.122449,1,1,1,26,6,1,1,4,4,1,59,1.0
2,33,170,60,20.761246,0,0,0,7,3,1,2,3,4,1,57,1.0
3,28,175,83,27.102041,2,1,2,11,2,1,0,5,2,3,40,1.0
4,32,183,80,23.888441,2,1,2,10,2,0,0,5,3,2,51,1.0


In [43]:
evening_data = df.loc[data['睡眠量表得分']<=42]
mid_data = df.loc[data['睡眠量表得分']>42]
mid_data = mid_data.loc[mid_data['睡眠量表得分']<58]
morning_data = df.loc[data['睡眠量表得分']>=58]
# morning_data = morning_data.reset_index()
# evening_data, mid_data, morning_data = pd.cut(x=data['睡眠量表得分'], bins=[0, 42, 58, ])

In [44]:
morning_data.columns#, mid_data.info(), evening_data.info()

Index(['年龄', '身高', '体重', 'BMI', '腰背痛程度评分', 'VAS评分', '椎间盘退变分数', 'Modic改变阴性椎体个数',
       'Modic改变阳性椎体个数', '许莫氏结节阴性椎体个数', '许莫氏结节阳性椎体个数', '睡眠量表得分', 'intercept',
       'smoke_无', 'smoke_有', 'alcohol_有', 'exercise_无', 'exercise_＜1次/周'],
      dtype='object')

In [54]:
data.describe()

,年龄,身高,体重,BMI,吸烟史,饮酒史,体育锻炼,腰背痛程度评分,VAS评分,椎间盘退变分数,Modic改变阴性椎体个数,Modic改变阳性椎体个数,许莫氏结节阴性椎体个数,许莫氏结节阳性椎体个数,睡眠量表得分,intercept
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.0
mean,29.533333,172.458333,68.675000,23.106728,0.550000,0.466667,1.275000,19.050000,4.633333,0.891667,3.108333,1.891667,4.216667,0.783333,51.108333,1.0
std,4.178644,4.953389,10.759528,3.598105,0.808048,0.500979,0.839793,7.296655,1.786661,0.312104,1.868788,1.868788,0.831960,0.831960,7.884455,0.0
min,20.000000,160.000000,49.000000,15.123457,0.000000,0.000000,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,37.000000,1.0
25%,26.000000,170.000000,62.000000,20.746716,0.000000,0.000000,0.750000,11.000000,3.000000,1.000000,2.000000,0.000000,4.000000,0.000000,49.500000,1.0
50%,30.000000,173.500000,70.000000,23.881778,0.000000,0.000000,2.000000,17.000000,5.000000,1.000000,4.000000,1.000000,4.000000,1.000000,51.000000,1.0
75%,33.000000,175.000000,75.000000,25.605536,1.000000,1.000000,2.000000,27.000000,6.000000,1.000000,5.000000,3.000000,5.000000,1.000000,57.500000,1.0
max,38.000000,183.000000,89.000000,30.863007,2.000000,1.000000,2.000000,32.000000,8.000000,1.000000,5.000000,5.000000,5.000000,3.000000,70.000000,1.0


In [ ]:
# sns.heatmap(data.iloc[:, :9].corr(), annot=True)
morning_data.corr()

In [ ]:
evening_data.corr()

# statistic 

In [51]:
x = df[x_cols]
y = df[['椎间盘退变分数']]
log_reg = sm.Logit(y, x)
log_fit = log_reg.fit()
print(log_fit.summary())

Optimization terminated successfully.
         Current function value: 0.308788
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                椎间盘退变分数   No. Observations:                  120
Model:                          Logit   Df Residuals:                      107
Method:                           MLE   Df Model:                           12
Date:                Wed, 10 Nov 2021   Pseudo R-squ.:                 0.09979
Time:                        15:39:49   Log-Likelihood:                -37.055
converged:                       True   LL-Null:                       -41.162
Covariance Type:            nonrobust   LLR p-value:                    0.7681
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
intercept        -26.1180     91.101     -0.287      0.774    -204.673     152.437
年龄               

In [56]:
x = data[['intercept', '年龄', '身高', '体重', 'BMI', '吸烟史', '饮酒史', '体育锻炼', '腰背痛程度评分', 'VAS评分', '睡眠量表得分']]
y = data[['椎间盘退变分数']]
log_reg = sm.Logit(y, x)
log_fit = log_reg.fit()
print(log_fit.summary())

Optimization terminated successfully.
         Current function value: 0.310758
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                椎间盘退变分数   No. Observations:                  120
Model:                          Logit   Df Residuals:                      109
Method:                           MLE   Df Model:                           10
Date:                Wed, 10 Nov 2021   Pseudo R-squ.:                 0.09404
Time:                        16:52:21   Log-Likelihood:                -37.291
converged:                       True   LL-Null:                       -41.162
Covariance Type:            nonrobust   LLR p-value:                    0.6540
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -30.4237     94.307     -0.323      0.747    -215.262     154.415
年龄            -0.0440      0.

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler() 
# morning_data_scaled =pd.DataFrame(scaler.fit_transform(morning_data[['年龄', '身高', '体重', 'BMI', '吸烟史', '饮酒史', '体育锻炼', '腰背痛程度评分', 'VAS评分', '睡眠量表得分']]), columns=['年龄', '身高', '体重', 'BMI', '吸烟史', '饮酒史', '体育锻炼', '腰背痛程度评分', 'VAS评分', '睡眠量表得分']) 

In [46]:
x_morn = morning_data[x_cols]
y_morn = morning_data[['椎间盘退变分数']]#.reset_index(drop=True)
log_reg_morn = sm.Logit(y_morn, x_morn)
log_fit_morn = log_reg_morn.fit()
print(log_fit_morn.summary())

         Current function value: inf
         Iterations: 35


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [60]:
data.duplicated().sum()

0

In [ ]:
# scaler = StandardScaler() 
# mid_data_scaled =pd.DataFrame(scaler.fit_transform(mid_data[['年龄', '身高', '体重', 'BMI', '吸烟史', '饮酒史', '体育锻炼', '腰背痛程度评分', 'VAS评分', '睡眠量表得分']]), columns=['年龄', '身高', '体重', 'BMI', '吸烟史', '饮酒史', '体育锻炼', '腰背痛程度评分', 'VAS评分', '睡眠量表得分']) 

In [52]:
x_mid = mid_data[x_cols]
y_mid = mid_data[['椎间盘退变分数']]
log_reg_mid = sm.Logit(y_mid, x_mid)
log_fit_mid = log_reg_mid.fit()
print(log_fit_mid.summary())

Optimization terminated successfully.
         Current function value: 0.313139
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                椎间盘退变分数   No. Observations:                   63
Model:                          Logit   Df Residuals:                       50
Method:                           MLE   Df Model:                           12
Date:                Wed, 10 Nov 2021   Pseudo R-squ.:                  0.2365
Time:                        15:40:48   Log-Likelihood:                -19.728
converged:                       True   LL-Null:                       -25.837
Covariance Type:            nonrobust   LLR p-value:                    0.4282
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
intercept       -133.2032    125.126     -1.065      0.287    -378.446     112.039
年龄               

In [49]:
x_evn = evening_data[x_cols]
y_evn = evening_data[['椎间盘退变分数']]
log_reg_evn = sm.Logit(y_evn, x_evn)
log_fit_evn = log_reg_evn.fit()
log_fit_evn.summary()

         Current function value: inf
         Iterations: 35


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [ ]:
# sns.pairplot(x)

# logistics regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
x = evening_data.iloc[:, :9].join(evening_data.iloc[:, -1])
y = evening_data.iloc[:, 9]
x_test_mid = mid_data.iloc[:, :9].join(mid_data.iloc[:, -1])
y_test_mid = mid_data.iloc[:, 9]
x_test_morn = morning_data.iloc[:, :9].join(morning_data.iloc[:, -1])
y_test_morn = morning_data.iloc[:, 9]

In [ ]:
log_eve=LogisticRegression()
log_eve.fit(x, y)
print('evening as baseline')
print('morining corelation r^2: ',log_eve.score(x_test_morn, y_test_morn),'\nmid corelation r^2: ',\
      log_eve.score(x_test_mid, y_test_mid), '\n截距b：', log_eve.intercept_, '\n回归系数', log_eve.coef_)

In [ ]:
log_mid=LogisticRegression()
log_mid.fit(x_test_mid, y_test_mid)
print('mid as baseline')
print('morining corelation r^2: ',log_mid.score(x_test_morn, y_test_morn),'\nevening corelation r^2: ',\
      log_mid.score(x, y), '\n截距b：', log_mid.intercept_, '\n回归系数', log_mid.coef_)

In [ ]:
log_morn=LogisticRegression()
log_morn.fit(x_test_morn, y_test_morn)
print('morning as baseline')
print('mid corelation r^2: ',log_morn.score(x_test_mid, y_test_mid),'\nevening corelation r^2: ',\
      log_morn.score(x, y), '\n截距b：', log_morn.intercept_, '\n回归系数', log_morn.coef_)